In [1]:
import tensorflow as tf
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

In [2]:
############################
# set up the run information
############################
root_directory   = '/glade/work/hardt/models'

model_run_name   = 'unet_v4p0'
model            = 'trained_model_feature-0to6.5km_at_500m_steps_label-3000m_2020_09_30_05_29.h5'
                    
output_data_name = 'predict_' + model + '.nc'

feature_data     = '/glade/work/hardt/ds612/2000-2013_June-Sept_QRAIN_INTERP_AGL_0to7km_at_500m_steps.nc'

BATCH_SIZE = 32
data_fraction_for_test = 0.10

############################

In [3]:
model_run_dir = os.path.join(root_directory, model_run_name)
output_data = os.path.join(model_run_dir, output_data_name)
input_model = os.path.join(model_run_dir, model)

if not os.path.isfile(input_model):
    print("ERROR: Missing input model file", input_model)
    sys.exit()

In [4]:
#
# set up the test data set
#
fds = xr.open_dataset(feature_data)
feature = fds.QRAIN.values

num_images = feature.shape[0]

test_data_start = int(num_images * (1 - data_fraction_for_test))
test_data_start = (num_images - int((num_images - test_data_start) / BATCH_SIZE) * BATCH_SIZE) 
test_data_end = num_images

print ()
print ("Number of images:", num_images)
print ("Test data start image:", test_data_start)
print ("Test data end image:", test_data_end)


Number of images: 9568
Test data start image: 8640
Test data end image: 9568


In [6]:
print(feature.shape)
feature = np.moveaxis(feature, 1, 3)
print(feature.shape)

(9568, 15, 256, 256)
(9568, 256, 256, 15)


In [7]:
test_dataset = tf.data.Dataset.from_tensor_slices((feature[test_data_start:test_data_end,:,:,:14]))
test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True)

In [9]:
print(test_dataset)

<BatchDataset shapes: (32, 256, 256, 14), types: tf.float32>
